In [9]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install OpenSSL

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement OpenSSL (from versions: none)
ERROR: No matching distribution found for OpenSSL


In [11]:
pip install oauth2client

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install --upgrade pyOpenSSL

  Using cached pyOpenSSL-24.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached cryptography-42.0.8-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
Using cached pyOpenSSL-24.1.0-py3-none-any.whl (56 kB)
Using cached cryptography-42.0.8-cp39-abi3-win_amd64.whl (2.9 MB)
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.7
    Uninstalling cryptography-3.4.7:
      Successfully uninstalled cryptography-3.4.7
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 20.0.0
    Uninstalling pyOpenSSL-20.0.0:
      Successfully uninstalled pyOpenSSL-20.0.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.


In [13]:
pip install cryptography==3.4.7

  Using cached cryptography-3.4.7-cp36-abi3-win_amd64.whl.metadata (5.1 kB)
Using cached cryptography-3.4.7-cp36-abi3-win_amd64.whl (1.6 MB)
Note: you may need to restart the kernel to use updated packages.  Attempting uninstall: cryptography
    Found existing installation: cryptography 42.0.8
    Uninstalling cryptography-42.0.8:
      Successfully uninstalled cryptography-42.0.8



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
apache-airflow 2.9.1 requires cryptography>=39.0.0, but you have cryptography 3.4.7 which is incompatible.
pyopenssl 24.1.0 requires cryptography<43,>=41.0.5, but you have cryptography 3.4.7 which is incompatible.


In [14]:
pip install PyOpenSSL==20.0.0

  Using cached pyOpenSSL-20.0.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached pyOpenSSL-20.0.0-py2.py3-none-any.whl (54 kB)
  Attempting uninstall: PyOpenSSL
    Found existing installation: pyOpenSSL 24.1.0
    Uninstalling pyOpenSSL-24.1.0:
      Successfully uninstalled pyOpenSSL-24.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from io import BytesIO

# Replace with your actual credentials (refer to Google Drive API documentation)
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = r"C:\Users\lokes\Downloads\dataventurex-project-63975-fbdce5cb55d3.json"  # Your service account key file
APPLICATION_NAME = 'Lead Processing Pipeline'

def get_authenticated_service():
    """Authenticates with Google Drive API using a service account and returns the Drive service object."""
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    return build('drive', 'v3', credentials=credentials)

def download_all_leads(service, folder_id):
    """Downloads all Excel files from the specified folder."""
    results = service.files().list(
        pageSize=1000,
        fields="files(id, name)",
        q=f"'{folder_id}' in parents and mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'"
    ).execute()
    items = results.get('files', [])
    if not items:
        print('No Excel files found in the folder.')
        return []
    
    downloaded_files = []
    for item in items:
        file_id = item['id']
        file_name = item['name']
        print(f"Downloading Excel file: {file_name}")
        
        request = service.files().get_media(fileId=file_id)
        file_content = request.execute()
        
        with open(file_name, 'wb') as f:
            f.write(file_content)
        downloaded_files.append(file_name)
    
    return downloaded_files

def process_excel_file(file_name):
    """Reads the Excel file, extracts relevant columns, and removes any empty rows."""
    df = pd.read_excel(file_name)
    df = df[['created_time', 'email', 'full_name', 'phone_number']]
    df['Message Status'] = 'No'
    df = df.dropna(how='all').reset_index(drop=True)
    return df

def consolidate_leads(lead_dfs):
    """Combines lead DataFrames from multiple files and keeps only unique records."""
    consolidated_df = pd.concat(lead_dfs, ignore_index=True)
    consolidated_df = consolidated_df.dropna(how='all').reset_index(drop=True)
    consolidated_df['unique_id'] = consolidated_df['full_name'].astype(str) + '_' + consolidated_df['phone_number'].astype(str) + '_' + consolidated_df['email'].astype(str)
    consolidated_df = consolidated_df.drop_duplicates(subset='unique_id', keep='first')
    consolidated_df = consolidated_df.drop(columns=['unique_id'])
    return consolidated_df

def create_output_files(service, consolidated_df, folder_id):
    """Updates existing contactList.xlsx and LeadStatus.xlsx files in Google Drive by appending new unique records and updating message status."""
    
    existing_files = service.files().list(
        q=f"'{folder_id}' in parents and (name='contactList.xlsx' or name='LeadStatus.xlsx')",
        fields="files(id, name)"
    ).execute().get('files', [])

    existing_files_dict = {file['name']: file['id'] for file in existing_files}

    def update_file(file_name, new_data_df):
        new_data_df = new_data_df.dropna()
        
        if file_name in existing_files_dict:
            request = service.files().get_media(fileId=existing_files_dict[file_name])
            file_content = request.execute()
            
            existing_data = pd.read_excel(BytesIO(file_content))
            existing_data = existing_data.drop(columns=['unique_id'], errors='ignore')
            existing_data = existing_data.dropna()
            
            if file_name == 'contactList.xlsx':
                existing_key = existing_data['full_name'].astype(str) + '_' + existing_data['phone_number'].astype(str)
                new_key = new_data_df['full_name'].astype(str) + '_' + new_data_df['phone_number'].astype(str)
            else:  # LeadStatus.xlsx
                existing_key = existing_data['created_time'].astype(str) + '_' + existing_data['email'].astype(str) + '_' + existing_data['full_name'].astype(str) + '_' + existing_data['phone_number'].astype(str)
                new_key = new_data_df['created_time'].astype(str) + '_' + new_data_df['email'].astype(str) + '_' + new_data_df['full_name'].astype(str) + '_' + new_data_df['phone_number'].astype(str)
            
            new_records = new_data_df[~new_key.isin(existing_key)]
            
            if file_name == 'LeadStatus.xlsx':
                # Update Message Status to 'Yes' for existing records
                existing_data.loc[:, 'Message Status'] = 'Yes'
            
            updated_data = pd.concat([existing_data, new_records], ignore_index=True)
        else:
            updated_data = new_data_df
        
        if file_name == 'contactList.xlsx':
            updated_data = updated_data[['full_name', 'phone_number']]
        else:  # LeadStatus.xlsx
            updated_data = updated_data[['created_time', 'email', 'full_name', 'phone_number', 'Message Status']]
        
        file_buffer = BytesIO()
        with pd.ExcelWriter(file_buffer, engine='openpyxl') as writer:
            updated_data.to_excel(writer, index=False)
        file_buffer.seek(0)

        media = MediaIoBaseUpload(file_buffer, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', resumable=True)

        if file_name in existing_files_dict:
            file = service.files().update(
                fileId=existing_files_dict[file_name],
                media_body=media
            ).execute()
            print(f"Updated {file_name} with ID: {file.get('id')}")
        else:
            file_metadata = {
                'name': file_name,
                'parents': [folder_id],
                'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
            }
            file = service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()
            print(f"Created new {file_name} with ID: {file.get('id')}")

    # Update contactList.xlsx
    contact_list_df = consolidated_df[['full_name', 'phone_number']].drop_duplicates()
    update_file('contactList.xlsx', contact_list_df)

    # Update LeadStatus.xlsx
    lead_status_df = consolidated_df[['created_time', 'email', 'full_name', 'phone_number', 'Message Status']]
    update_file('LeadStatus.xlsx', lead_status_df)

def main():
    """Main function to execute the pipeline."""
    service = get_authenticated_service()
    # Replace with your actual input folder ID
    input_folder_id = '1jIIRsdyob6MiRA5BXhVepn7auZDPrEgO'
    # Replace with your actual output folder ID
    output_folder_id = '1xbzWWNa9oWkHLiILAC0PvGiXozBg7NXV'
    downloaded_files = download_all_leads(service, input_folder_id)
    if downloaded_files:
        lead_dfs = []
        for file in downloaded_files:
            lead_dfs.append(process_excel_file(file))
        consolidated_df = consolidate_leads(lead_dfs)
        create_output_files(service, consolidated_df, output_folder_id)
        print('Pipeline completed successfully!')
    else:
        print('No files were downloaded. Pipeline could not be completed.')

if __name__ == '__main__':
    main()

Updated contactList.xlsx with ID: 1a2rbictj-D0bkBxBqEbzM2vZ6tdU8Q6P
Updated LeadStatus.xlsx with ID: 1tBJNnTmF83qqzieKy6iQexOkzKhK9ba9
Pipeline completed successfully!
